In [ ]:
import sympy as smp
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
from pathlib import Path
from tqdm import tqdm

## Coordinates

Let a pendulum of length $L$ have a non-fixed pivot oscillating sinusoidally along the $y$-axis with amplitude $A$ and frequency $\nu$. The arm is massless with a bob on the end of mass $m$. The position of the bob is

$$
x=L\cos{\theta} \\
y=A\sin{(2\pi\nu t)}+L\sin{\theta} \\
$$

In [ ]:
t, g, m, A, L, v = smp.symbols('t, g, m, A, L, nu')

o = smp.symbols('theta', cls=smp.Function)
o = o(t)

In [ ]:
x = L*smp.cos(o)
y = A*smp.sin(2*smp.pi*v*t) + L*smp.sin(o)

x_d = smp.diff(x, t)
y_d = smp.diff(y, t)

o_d = smp.diff(o, t)
o_dd = smp.diff(o_d, t)

In [ ]:
T = smp.Rational(1,2)*m*(x_d**2+y_d**2)
V = m*g*y
L = T - V
L.simplify()

In [ ]:
leq = smp.diff(L, o) - smp.diff(smp.diff(L, o_d), t)
o_dd_ode = smp.solve([leq],[o_dd])[o_dd].simplify()
o_dd_ode

## Equation of Motion

$$
\ddot{\theta}=(\frac{4\pi^{2}A\nu^{2}}{L}\sin{(2\pi\nu t)}-\frac{g}{L})\cos{\theta}
$$

### Dimensions

Let 

$$
r_{A}=\frac{A}{L} \\
r_{g}=\frac{g}{L\nu^{2}} \\
$$

$$
\frac{\ddot{\theta}}{\nu^{2}}=(4\pi^{2}r_{A}\sin{(2\pi\nu t)}-r_{g})\cos{\theta}
$$

This will measure the amplitude of the vertical oscillations in terms of the length of the pendulum and the gravitational acceleration in terms of the pendulum length and frequency of vertical oscillations. The equation of motion is also now dimensionless, and the simplest equation measuring time in terms of $\nu$ is

$$
\ddot{\theta}=(4\pi^{2}r_{A}\sin{(2\pi t)}-r_{g})\cos{\theta}
$$

## Integration

Let $\omega = \dot{\theta}$.

$$
\frac{d\theta}{dt}=\omega \\
\frac{d\omega}{dt}=(4\pi^{2}r_{A}\sin{(2\pi t)}-r_{g})\cos{\theta} \\
$$

Let $\vec{S}=(\theta,\omega)$

In [ ]:
r_a = 0.1  # A in terms of L
r_g = 0.5 # g in terms of A and v

t_f = 100   # total amount of oscilation periods
res = 1000 # samples per period
fps = 50   # animation framerate

t = np.linspace(0, t_f, res*t_f)

def dSdt(S, t):
    o, o_d = S
    return [
        o_d,
        (4*np.pi**2*r_a*np.sin(2*np.pi*t)-r_g)*np.cos(o)
    ]

sol = odeint(dSdt,[np.pi/2.1,0],t)

o_vals = sol.T[0]
w_vals = sol.T[1]

plt.plot(t, o_vals)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.grid()
bound = 1.5*(r_a+1)
ax.set_xlim(-bound, bound)
ax.set_ylim(-bound, bound)

ln1, = plt.plot([],[], 'ro-')
time_text = ax.text(0.05, 0.90, '', transform=ax.transAxes, fontsize=20)

ani_time = 10

def animate(i):
    t = (t_f/ani_time)*(i/fps)
    ln1.set_data(
        [0,                       np.cos(o_vals[int(t*res)])],
        [r_a*np.sin(2*np.pi*v*t), r_a*np.sin(2*np.pi*v*t)+np.sin(o_vals[int(t*res)])]
    )
    time_text.set_text(fr'$\nu t$ = {np.round(t,2)}')

ani = FuncAnimation(fig, animate, frames=fps*10, interval=1000/fps)
ani.save(Path('docs') / Path('kaptiza_pendulum.gif'), writer='pillow', fps=fps, dpi=100)